# 1.4 - External Data Collection - web-scraping interest ontology from wikipedia.org

In [332]:
from bs4 import BeautifulSoup
import bs4
import requests
import pandas as pd

## 1. Download Outline - Academic Disciplines

- #### 1.1 Store in flat table, STOP after 5th Level

    *Hack Approach*:
    - Pre-set levels (1-Category (h2), 2-Sub-category (h3), 3-Topic, 4-Sub-Topic, 5-Sub-Topic, Page)
    - One for each page (including headers)
    - If (ul, li) go deeper
    

- #### 1.2 Clean unnecessary cols

In [359]:
##TODO : Add to pandas dataframe #######

#Digest Wikipedia Academic Disciplines Outline Script ####################################################################################################

#HELPER FX  #####################################################################
def get_header_name_page(el, blacklist=[]):    
    name = ''
    page_href = ''
    
    span = el.find('span', class_='mw-headline')
    if span != None and span.text not in blacklist: #skip non-category headers
            name = span.text
            a = span.find('a')
            if a != None:
                page_href = a['href']
            else:
            # grab article ref from matching main article
                candidate_articles = span.find_next('div').find_all('a')
                page_href = [article['href'] for article in candidate_articles]
    return name, page_href

#################################################################################

wikiPagePaths = []

#0. Get outline
url = "https://en.wikipedia.org/wiki/Outline_of_academic_disciplines"
r = requests.get(url)
soup = BeautifulSoup(r.content)

#1 - start unpacking at first category (h2)
start_of_outline = soup.select("h2 span", class_='mw-headline')[0].parent
outline = [start_of_outline]
outline.extend(list(start_of_outline.next_siblings))

category_h2_1 = sub_category_h3_2 = topic_3 = subtopic_4 = subtopic_5 = ''

#2 - grab key outline elements (h2, h3, h4, and bulleted subtopics)
for el in outline:
    #2.1 - grab header elements (h2, h3, h4)
    
    if el.name == 'h2':
        #skip blacklisted elements
        if el.find('span', class_='mw-headline').text in ['See also', 'References', 'External links']:
            continue

        category_h2_1, page_href = get_header_name_page(el)        #get category metadata
        sub_category_h3_2 = topic_3 = subtopic_4 = subtopic_5 = '' #clear categories previous metadata
        
        print(category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href)
        wikiPagePaths.append([category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href])
        continue
            
    elif el.name == 'h3':

        sub_category_h3_2, page_href = get_header_name_page(el) #get subcategory metadata
        topic_3 = subtopic_4 = subtopic_5 = ''                  #clear categories previous metadata
        
        print(category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href)
        wikiPagePaths.append([category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href])
        continue
        
    elif el.name == 'h4':

        topic_3, page_href = get_header_name_page(el) #get subcategory metadata
        subtopic_4 = subtopic_5 = ''                  #clear categories previous metadata
        
        print(category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href)
        wikiPagePaths.append([category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href])
        continue
        
    #2.2 - grab nested bulleted lists of topics and subtopics
    elif el.name == 'div' and 'div-col' in el.get('class'):
        ul = el.find('ul')
        for li in ul:
            if isinstance(li, bs4.element.Tag):
                subtopic_4 = li.find('a').text
                page_href = li.find('a')['href']
                print(category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href)
                wikiPagePaths.append([category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href])
            
            
            ul = li.find('ul')
            if ul != -1 and ul != None:
                for li in ul:
                    if isinstance(li, bs4.element.Tag):
                        subtopic_5 = li.find('a').text
                        page_href = li.find('a')['href']
                        print(category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href)
                        wikiPagePaths.append([category_h2_1, sub_category_h3_2, topic_3, subtopic_4, subtopic_5, page_href])
        
    else:
        continue

wikiPagePathsDF = pd.DataFrame(index=['category_h2_1', 'sub_category_h3_2', 'topic_3', 'subtopic_4', 'subtopic_5', 'page_href'], col=wikiPagePaths)

Humanities     /wiki/Humanities
Humanities Arts    /wiki/The_arts
Humanities Arts Performing arts   /wiki/Performing_arts
Humanities Arts Performing arts Music  /wiki/Music
Humanities Arts Performing arts Music Accompanying /wiki/Accompaniment
Humanities Arts Performing arts Music Chamber music /wiki/Chamber_music
Humanities Arts Performing arts Music Church music /wiki/Church_music
Humanities Arts Performing arts Music Conducting /wiki/Conducting
Humanities Arts Performing arts Music Early music /wiki/Early_music
Humanities Arts Performing arts Music Jazz studies /wiki/Jazz
Humanities Arts Performing arts Music Musical composition /wiki/Musical_composition
Humanities Arts Performing arts Music Music education /wiki/Music_education
Humanities Arts Performing arts Music Music history /wiki/Music_history
Humanities Arts Performing arts Music Musicology /wiki/Musicology
Humanities Arts Performing arts Music Ethnomusicology /wiki/Ethnomusicology
Humanities Arts Performing arts Music Music 

## 2. Crawl Outline and Download Wiki articles

- ### 2.1 Download article text from outline

In [ ]:
#2.1.0 - Setup scraper through API
wiki_wiki = wikipediaapi.Wikipedia(
            language='en',
            extract_format=wikipediaapi.ExtractFormat.WIKI
    )

# For article in wikiPagePathDF
for page in wikiPagePathDF.axis(1):
    
    #2.1.1 Request page
#     url = "https://en.wikipedia.org/wiki/{page_href}".format(page_href=page.page_href)
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content)

    p_wiki = wiki_wiki.page(page_href)
    print(p_wiki.text)
    
    #2.1.2 add to col in wikiPagePathDF  
    #TODO - keep everything??
    wikiPagePathDF['article_bag_of_words'] = p_wiki.text